In [1125]:

# pip3 install cloudscraper beautifulsoup4
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import time
import re
from deep_translator import GoogleTranslator
from unidecode import unidecode
import json
from datetime import datetime, timedelta
import random
import requests
from langdetect import detect

In [1126]:
import numpy as np
# Hàm để xử lý kiểu dữ liệu không serializable
def default_converter(o):
    if isinstance(o, (np.integer, np.floating)):  # Xử lý kiểu số NumPy
        return int(o)
    elif isinstance(o, (np.ndarray,)):           # Xử lý mảng NumPy (nếu có)
        return o.tolist()
    return str(o)  # Xử lý các kiểu dữ liệu khác

In [1127]:
def convert_to_camel_case(text):
    # Chuyển tất cả chữ về thường
    text = unidecode(text.lower())
    # Tách các từ dựa trên khoảng trắng
    words = text.split()
    # Viết hoa chữ cái đầu của từng từ và nối chúng lại
    camel_case = ''.join(word.capitalize() for word in words)
    return camel_case

In [1128]:
def extract_number(text):
    # Tìm tất cả các số trong chuỗi
    numbers = re.findall(r'\d+', text)
    # Nếu có số, trả về số đầu tiên (hoặc có thể thay đổi để trả về tất cả nếu cần)
    return int(numbers[0]) if numbers else None

In [1129]:
def get_url(soup):
    canonical_tag = soup.find('link', rel='canonical')
    # Lấy giá trị của thuộc tính href
    if canonical_tag and 'href' in canonical_tag.attrs:
        canonical_url = canonical_tag['href']
        return canonical_url
    return None

In [1130]:
def get_url_id(soup):
    url = get_url(soup)
    # Regex để tìm ID trong URL
    match = re.search(r'-(\d+)-jv', url)
    if match:
        return match.group(1)  # Trả về ID tìm thấy
    return None

In [1131]:
def get_heading(soup):
    heading1 = soup.find('h1', class_ = "sc-ab270149-0 hAejeW")
    if heading1:
        content = heading1.get_text(separator=' ', strip=True)
        return content
    return None

In [1132]:
def get_salary(soup):
    heading1 = soup.find('span', attrs={"name": "label"})
    if heading1:
        content = heading1.get_text(separator=' ', strip=True)
        return content
    return None

In [1133]:
def get_han_nop_ho_so(soup):
    data = soup.find('div', class_="sc-2557ebc-1 iwszkh")
    if data:
        # Ngày hiện tại (chỉ ngày tháng năm)
        today = datetime.now().date()
        # Cộng ngày
        new_date = today + timedelta(days=extract_number(data.text))
        formatted_new_date = new_date.strftime("%d/%m/%Y")
        return formatted_new_date
    return None

In [1134]:
def get_vi_tri(soup):
    data = soup.find('div', class_="sc-2557ebc-1 ebdjLi")
    if data:
        return data.text
    return None

In [1135]:
def get_thong_tin_cv(soup):
    data = soup.find_all('div', class_="sc-1671001a-6 dVvinc")
    if data[0]:
        mo_ta = data[0].div.decode_contents().strip()
    else:
        mo_ta = None
    if data[1]:
        yeu_cau = data[1].div.decode_contents().strip()
    else:
        yeu_cau = None
    return mo_ta, yeu_cau

In [1136]:
def get_dia_chi(soup):
    array = []
    data = soup.find_all('div', class_="sc-8868b866-0 dBRwI")
    if data:
        for temp in data:
            array.append(temp.text)
        return array
    return None

In [1137]:
def get_keyword(soup):
    # Tìm tất cả các thẻ button có class chứa những lớp cần tìm
    data = soup.find_all('button', class_=lambda class_list: class_list and 
                         all(c in class_list for c in ["sc-db4ea809-0", "jpWTwo", "btn-default", "btn-sm"]))
    
    array = []
    if data:
        for temp in data:
            array.append(temp.text.strip())  # strip() để loại bỏ khoảng trắng thừa
        return array
    return None


In [1138]:
def get_phuc_loi(soup):
    data = soup.find_all('div', class_="sc-8868b866-0 hoIaMz")
    dict = {}
    if data:
        for temp in data:
            if temp.find('div', class_ = "sc-c683181c-2 fGxLZh"):
                dict[convert_to_camel_case(temp.find('p', class_ = "sc-ab270149-0 jlpjAq").text)] = (temp.find('div', class_ = "sc-c683181c-2 fGxLZh").text)
                # array.append(dict)
        return dict
    return None

In [1139]:
def get_id_cong_ty(url):
    # Biểu thức chính quy tìm kiếm mã ID sau 'c'
    match = re.search(r'c(\d+)', url)
    if match:
        return match.group(1)  # Trả về kết quả khớp hoàn toàn 'c311795'
    return None

In [1140]:
def get_ten_cong_ty(soup):
    id = None
    url = None
    name = None
    data = soup.find('a', class_="sc-ab270149-0 egZKeY sc-f0821106-0 gWSkfE")
    if data:
        url = data.get('href')
        name = data.text
        if url:
            id = get_id_cong_ty(url)
    return url, id, name

In [1141]:
def get_thong_tin_cong_ty(soup):
    array = []
    delete = soup.find('a', class_="sc-ab270149-0 qYMDb sc-f0821106-0 gWSkfE")
    if delete:
        delete.decompose()
    data = soup.find_all('div', class_="sc-37577279-5 kQCIWi")
    if data:
        for temp in data:
            array.append(temp.text)
        if len(array) == 2:
            array.append(array[1])
            array[1] = None
        if len(array) == 1:
            array.append(None)
            array.append(None)
        return array
    return [None, None, None]

In [1142]:
def convert_number(value):
    # try:
        if not value:
            return None
        number = float(value.replace(",", ""))
        # Kiểm tra nếu là số nguyên
        if number.is_integer():
            return int(number)
        else:
            return number  # Trả về float
    # except Exception:
    #     return value  # Trả về giá trị gốc nếu có lỗi

In [1143]:
def convert_value(value):
    if not value or value == 'None':
        return None
    try:
        # Kiểm tra nếu là số nguyên
        return int(value)
    except ValueError:
        try:
            # Kiểm tra nếu là số thập phân
            return float(value)
        except ValueError:
            # Nếu không phải số, trả về dưới dạng chuỗi
            return str(value)


In [1144]:
def get_thong_tin_viec_lam(soup):
    data = soup.find_all('div', class_="sc-7bf5461f-2 JtIju")
    dict = {}
    if data:
        for temp in data:
            dict[convert_to_camel_case(temp.find('label', class_ = "sc-ab270149-0 dfyRSX").text)] = (temp.find('p', class_ = "sc-ab270149-0 cLLblL").text)
            # array.append(dict)
        dict['SoNamKinhNghiemToiThieu'] = convert_value(dict['SoNamKinhNghiemToiThieu'])
        return dict
    return None

In [1145]:
def extract_number_salary(text):
    # Biểu thức chính quy tìm kiếm số, bao gồm số có dấu phẩy phân cách
    match = re.search(r'[\d,]+', text)
    if match:
        # Loại bỏ dấu ',' và trả về số dưới dạng float
        number = match.group(0)
        return convert_number(number)  # Chuyển đổi thành float
    return None

In [1146]:
def normalize_text(text):
    # Các từ cần loại bỏ
    text = str(text)
    text = re.sub(r"\bTP\.?\s*", "", text, flags=re.IGNORECASE)
    words_to_remove = ["huyện", "thị xã", "thành phố", "quận", "tỉnh"]
    
    # Loại bỏ các từ không cần thiết
    for word in words_to_remove:
        text = text.lower().replace(word, "")
    
    # Loại bỏ dấu tiếng Việt
    text = unidecode(text)
    # Loại bỏ dấu cách và chuyển về chữ thường
    text = text.replace(" ", "").lower()
    return text


In [1147]:
def load_df():
    return pd.read_excel(r'C:\Users\My_Pc\Desktop\Thư Viện Pháp Luật final\Crawl tuyển dụng\vietnamwork_data_setup.xlsx', engine='openpyxl')

In [1148]:
# Tìm kiếm với nhiều điều kiện
def find_id_tinhThanh(name, df):
    # print(name)
    result = df[
        (df['processing'].apply(lambda x: str(x).lower() in normalize_text(name.lower()))) &
        (df['ParentId'] == 0) &
        (df['Level'] == 1)
    ]

    if not result.empty:
        id_value = result.iloc[0]['Id']
        # parent_id_value = result.iloc[0]['ParentId']
        # level_value = result.iloc[0]['Level']
        name_value = result.iloc[0]['UnitName']
        name_value_processing = result.iloc[0]['processing']
        # print(f"Id: {id_value}, ParentId: {parent_id_value}, Level: {level_value}")
        return id_value, name_value, name_value_processing
    return None, None, None


In [1149]:
# Tìm kiếm với nhiều điều kiện
def find_id_tinhQuanHuyen(name, ParentId, df):
    if not name:
        return None, None, None
    name = re.sub(r"Quận 2", "Thủ Đức", name, flags=re.IGNORECASE)
    # name = re.sub(r"\bTP\.?\s*", "", name, flags=re.IGNORECASE)
    result = df[
        (df['processing'].apply(lambda x: str(x).lower() in normalize_text(name.lower()))) &
        (df['ParentId'] == ParentId) &
        (df['Level'] == 2)
    ]

    if not result.empty:
        id_value = result.iloc[0]['Id']
        name_value = result.iloc[0]['UnitName']
        name_value_processing = result.iloc[0]['processing']
        # print(f"Id: {id_value}, ParentId: {parent_id_value}, Level: {level_value}")
        return id_value, name_value, name_value_processing
    return None, None, None


In [1150]:
def split_address(dia_chi_input):
    if ',' in dia_chi_input:
        dia_chi_input_split = dia_chi_input.split(',')
    elif '-' in dia_chi_input:
        dia_chi_input_split = dia_chi_input.split('-')
    elif '–' in dia_chi_input:
        dia_chi_input_split = dia_chi_input.split('–')
    else:
        # Nếu không có dấu phẩy và dấu gạch ngang, trả về chuỗi ban đầu trong danh sách
        dia_chi_input_split = [dia_chi_input]
    
    # Loại bỏ khoảng trắng thừa cho mỗi phần tử
    dia_chi_input_split = [item.strip() for item in dia_chi_input_split]
    
    return dia_chi_input_split

In [1151]:
def remove_from_last_match(word, word_list):
    # Duyệt từ cuối mảng về đầu để tìm từ trùng khớp cuối cùng
    for i in range(len(word_list) - 1, -1, -1):
        # print(normalize_text(word))
        # print(normalize_text(word_list[i]))
        if str(normalize_text(word)) in normalize_text(word_list[i]):
            # Cắt mảng từ vị trí i trở đi
            return word_list[:i]
    
    # Nếu không có từ trùng khớp, trả về mảng gốc
    return word_list

In [1152]:
def is_english(text):
    try:
        # Nhận diện ngôn ngữ của văn bản
        return detect(text) == 'en'
    except:
        return False


In [1153]:
def process_address_data(locations, dia_chi_inputs, df):
    if not locations or not dia_chi_inputs:
        return None
    locations = locations.split(',')
    output = []
    output_tu_nhap = []
    for location, dia_chi_input in zip(locations, dia_chi_inputs):
        # print(find_id_tinhThanh(location, df))
        tinhThanh = find_id_tinhThanh(location, df)
        con_lai = remove_from_last_match(tinhThanh[-1], split_address(dia_chi_input))
        if len(con_lai) >= 1:
            tinhQuanHuyen = (find_id_tinhQuanHuyen(con_lai[-1], tinhThanh[0], df))
            if tinhQuanHuyen[-1]:
                con_lai = (remove_from_last_match(tinhQuanHuyen[-1], (con_lai)))
                diaChiDict = {
                    'IdTinhThanh': tinhThanh[0],
                    'TinhThanh': tinhThanh[1],
                    'IdQuanHuyen': tinhQuanHuyen[0],
                    'TinhQuanHuyen':tinhQuanHuyen[1],
                }
                data_con_lai = ','.join(con_lai)
                if data_con_lai:
                    diaChiDict['DiaChi'] = data_con_lai
                    diaChiDict['IsEN'] = is_english(dia_chi_input)
                else:
                    diaChiDict['DiaChi'] = None
                    diaChiDict['IsEN'] = is_english(dia_chi_input)
            else:
                diaChiDict = {
                    'IdTinhThanh': tinhThanh[0],
                    'TinhThanh': tinhThanh[1],
                    'IdQuanHuyen': None,
                    'TinhQuanHuyen': None,
                    'DiaChi': ','.join(con_lai),
                    'IsEN': is_english(dia_chi_input),
                }
        else:
                diaChiDict = {
                'IdTinhThanh': tinhThanh[0],
                'TinhThanh': tinhThanh[1],
                'IdQuanHuyen': None,
                'TinhQuanHuyen':None,
                'DiaChi': None,
                'IsEN': is_english(dia_chi_input),
            }
        if tinhThanh[0] == None:
            output_tu_nhap.append(dia_chi_inputs)       
        else: 
            output.append(diaChiDict)
        dict_final = {
            'DiaDiemCuThe': output,
            'DiaDiemTuNhap': output_tu_nhap,
        }
    return dict_final

In [1154]:
def processing_salary(text):
    muc_luong = text.rstrip('/tháng')
    if '$' in muc_luong:
        don_vi_tien = 'USD'
        muc_luong = muc_luong.replace('$', '')
    elif '₫' in muc_luong:
        don_vi_tien = 'VND'
        muc_luong = muc_luong.replace('₫', '')
    elif '¥' in muc_luong:
        don_vi_tien = 'JPY'
        muc_luong = muc_luong.replace('¥', '')
    else:
        don_vi_tien = None
    muc_luong = muc_luong.split('-')
    if len(muc_luong) > 1:
        muc_luong_den = extract_number_salary(muc_luong[1].strip())
        muc_luong_tu = extract_number_salary(muc_luong[0].strip())
    else:
        if muc_luong[0].strip().lower().startswith('trên') or muc_luong[0].strip().lower().startswith('từ'):
            muc_luong_den = 0
            muc_luong_tu = extract_number_salary(muc_luong[0])
        elif muc_luong[0].strip().lower().startswith('dưới') or muc_luong[0].strip().lower().startswith('tới'):
            muc_luong_tu = 0
            muc_luong_den = extract_number_salary(muc_luong[0])
        else:
            muc_luong_tu = 0
            muc_luong_den = 0
    return muc_luong_tu, muc_luong_den, don_vi_tien

In [1155]:
def extract_redirect_url(html_content):
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Tìm thẻ meta có thuộc tính http-equiv="refresh"
    meta_tag = soup.find('meta', attrs={'http-equiv': 'refresh'})
    
    if meta_tag and 'content' in meta_tag.attrs:
        # Tách URL từ content của thẻ meta
        content = meta_tag['content']
        if 'url=' in content:
            return content.split('url=')[-1]  # Trả về URL sau "url="
    return None  # Trả về None nếu không tìm thấy URL

In [1156]:
def IsThoaThuan(text):
    if processing_salary(text) == (0, 0, None):
        return True
    return False

In [ ]:
df = load_df()
output = []
for temp in range(0,100):
    while True:
        try:
            ids = 1846936 + temp
            scraper = cloudscraper.create_scraper(
                            browser={
                                "browser": "chrome",
                                "platform": "windows",
                            },
                        )
            # specify the target URL
            url = f"https://www.vietnamworks.com/nhan-vien-ban-hang-outsource--{ids}-jv"
            response = scraper.get(url)
            url = extract_redirect_url(response.text)
            response = scraper.get(url)
            print(response.status_code, ids)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                if get_heading(soup) and convert_value(get_ten_cong_ty(soup)[1]):
                    data = {
                        'JobId': convert_value(get_url_id(soup)),
                        'JobTitle': convert_value(get_heading(soup)),
                        'UrlJob': convert_value(url),
                        'IsThoaThuan': IsThoaThuan(get_salary(soup)),
                        'mucluongfrom': processing_salary(get_salary(soup))[0],
                        'mucluongto': processing_salary(get_salary(soup))[1],
                        'currency': processing_salary(get_salary(soup))[2],
                        'DiaDiem': convert_value(get_vi_tri(soup)),
                        'HanNopHoSo': convert_value(get_han_nop_ho_so(soup)),
                        'MoTaCongViec': convert_value(get_thong_tin_cv(soup)[0]),
                        'YeuCauUngVien': convert_value(get_thong_tin_cv(soup)[1]),
                        'QuyenLoi': get_phuc_loi(soup),
                        'DiaDiemLamViec': process_address_data(convert_value(get_vi_tri(soup)), get_dia_chi(soup), df),
                        'ThongTinViecLam': get_thong_tin_viec_lam(soup),
                        'IdCongTy': convert_value(get_ten_cong_ty(soup)[1]), 
                        'UrlCongTy': convert_value(get_ten_cong_ty(soup)[0]),
                        'TenCongTy': convert_value(get_ten_cong_ty(soup)[2]),
                        'DiaChiCongTy': convert_value(get_thong_tin_cong_ty(soup)[0]),
                        'QuyMo': convert_value(get_thong_tin_cong_ty(soup)[1]),
                        'NguoiLienHe': convert_value(get_thong_tin_cong_ty(soup)[2]),
                        'TuKhoa': get_keyword(soup),
                    }
                    output.append(data)
                    print(json.dumps(data, ensure_ascii=False, indent=5, default=default_converter))
                    print(response.status_code, "Cập nhật thông tin thành công.", ids, len(output))
            if response.status_code == 429:
                    # print(f"ID {ids}: Too many requests, retrying...")
                time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
            else:
                break  # Thoát khỏi vòng lặp nếu không phải 429
        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            print(f"ID {ids}: Lỗi kết nối {e}. Đang thử lại...")
            time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại


200 1846936
{
     "JobId": 1846936,
     "JobTitle": "Tekla Modeler",
     "UrlJob": "https://www.vietnamworks.com/tekla-modeler--1846936-jv",
     "IsThoaThuan": true,
     "mucluongfrom": 0,
     "mucluongto": 0,
     "currency": null,
     "DiaDiem": "Hồ Chí Minh",
     "HanNopHoSo": "27/12/2024",
     "MoTaCongViec": "<p><strong>Thornton Tomasetti</strong> is seeking multiple talented Tekla Modelers for its Vietnam office.</p><p><br/></p><p><strong>Thornton Tomasetti </strong>applies engineering and scientific principles to solve the world’s challenges. An independent organization of creative thinkers and innovative doers collaborating from offices worldwide, our mission is to bring our clients’ ideas to life and in the process, lay the groundwork for a better, more resilient future. We provide support and opportunities to our employees to achieve their full potential and cultivate a rewarding career.</p><p><br/></p><p><strong>Our Construction Engineering </strong>Practice leverag

In [1158]:

# df = load_df()
# output = []
# for temp in range(0,100):
#     while True:
#         try:
#             ids = 1846996 + temp
#             scraper = cloudscraper.create_scraper(
#                             browser={
#                                 "browser": "chrome",
#                                 "platform": "windows",
#                             },
#                         )
#             df = load_df()
#             # specify the target URL
#             url = f"https://www.vietnamworks.com/nhan-vien-ban-hang-outsource--{ids}-jv"
#             response = scraper.get(url)
#             url = extract_redirect_url(response.text)
#             response = scraper.get(url)
#             print(response.status_code)
#             if response.status_code == 200:
#                 soup = BeautifulSoup(response.text, 'html.parser')
#                 if get_heading(soup) and convert_value(get_ten_cong_ty(soup)[1]):
#                     data = {
#                         'JobId': convert_value(get_url_id(soup)),
#                         'JobTitle': convert_value(get_heading(soup)),
#                         'UrlJob': convert_value(url),
#                         'IsThoaThuan': IsThoaThuan(get_salary(soup)),
#                         'mucluongfrom': processing_salary(get_salary(soup))[0],
#                         'mucluongto': processing_salary(get_salary(soup))[1],
#                         'currency': processing_salary(get_salary(soup))[2],
#                         'DiaDiem': convert_value(get_vi_tri(soup)),
#                         'HanNopHoSo': convert_value(get_han_nop_ho_so(soup)),
#                         'MoTaCongViec': convert_value(get_thong_tin_cv(soup)[0]),
#                         'YeuCauUngVien': convert_value(get_thong_tin_cv(soup)[1]),
#                         'QuyenLoi': get_phuc_loi(soup),
#                         'DiaDiemLamViec': process_address_data(convert_value(get_vi_tri(soup), get_dia_chi(soup), df),
#                         'ThongTinViecLam': get_thong_tin_viec_lam(soup),
#                         'IdCongTy': convert_value(get_ten_cong_ty(soup)[1]), 
#                         'UrlCongTy': convert_value(get_ten_cong_ty(soup)[0]),
#                         'TenCongTy': convert_value(get_ten_cong_ty(soup)[2]),
#                         'DiaChiCongTy': convert_value(get_thong_tin_cong_ty(soup)[0]),
#                         'QuyMo': convert_value(get_thong_tin_cong_ty(soup)[1]),
#                         'NguoiLienHe': convert_value(get_thong_tin_cong_ty(soup)[2]),
#                         'TuKhoa': get_keyword(soup),
#                     }
#                     output.append(data)
#                     # print(data)
#                     print(response.status_code, "Cập nhật thông tin thành công.", ids, len(output))
#             if response.status_code == 429:
#                     # print(f"ID {ids}: Too many requests, retrying...")
#                 time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
#             else:
#                 break  # Thoát khỏi vòng lặp nếu không phải 429
#         except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
#             print(f"ID {ids}: Lỗi kết nối {e}. Đang thử lại...")
#             time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại

In [1159]:
# # Chuyển mảng thành DataFrame
# df = pd.DataFrame({'địa chỉ': output})

# # Lưu DataFrame vào file Excel
# file_path = 'dia_chi.xlsx'
# df.to_excel(file_path, index=False, engine='openpyxl')

# print(f"Dữ liệu đã được lưu vào file {file_path}")

In [1161]:
# # Lưu vào tệp JSON
with open("sample_vietnamworks_28_11_2024_2.json", "w", encoding="utf-8") as json_file:
    json.dump(output, json_file, ensure_ascii=False, indent=5, default=default_converter)